In [1]:
from pprint import pprint
from typing import Callable
import librosa
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers.pipelines.automatic_speech_recognition import AutomaticSpeechRecognitionPipeline

torch.set_float32_matmul_precision("medium")

/home/demontego/whisper-on-speed/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_float32_matmul_precision("medium")
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3-turbo"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, attn_implementation='flash_attention_2',
).eval().cuda()
# model.generation_config.cache_implementation = "static"
# model.generation_config.max_new_tokens = 324
# model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [3]:
processor = AutoProcessor.from_pretrained(model_id)
# pipe = AutomaticSpeechRecognitionPipeline(
#     model=model,
#     tokenizer=processor.tokenizer,
#     feature_extractor=processor.feature_extractor,
#     torch_dtype=torch.float16,
#     stride_length_s=0.3,
#     chunk_length_s=10,
#     device=device,
# )

In [4]:
audio, _ = librosa.load("/home/demontego/whisper/tests/test_data/output.wav", sr=16000)

In [5]:
len(audio) / 16000

178.432

In [6]:
import numpy as np


def chunk_audio_with_overlap(
    audio: np.ndarray,  # (n_samples,)
    vad: torch.nn.Module,
    get_speech_timestamps: Callable,
    sr: int = 16000,
    chunk_sec: int = 10,
    overlap_sec: float = 0.5
) -> np.ndarray[np.ndarray[np.float16]]:  # (N_chunks, chunk_len)
    """
    Режет аудиомассив на куски с перекрытием и возвращает массив чанков.

    :param audio: 1D np.ndarray аудиосигнала (float32 или int16)
    :param sr: sample rate (по умолчанию 16kHz)
    :param chunk_sec: длина каждого чанка в секундах
    :param overlap_sec: длина перекрытия в секундах
    :return: np.ndarray формы (N_chunks, chunk_len) с dtype float32
    """
    if audio.ndim != 1:
        raise ValueError("Input audio must be a 1D np.ndarray.")

    # Приводим к float32
    audio = audio.astype(np.float16)

    chunk_len = int(sr * chunk_sec)
    step = int(chunk_len - sr * overlap_sec)
    total_len = len(audio)

    chunks = []
    seconds = []

    for start in range(0, total_len, step):
        end = start + chunk_len
        chunk = audio[start:end]
        if len(chunk) < chunk_len:
            chunk = np.pad(chunk, (0, chunk_len - len(chunk)))
        speech_timestamps = get_speech_timestamps(chunk, vad, return_seconds=True)
        sum_talk_seconds = sum(item['end'] - item['start'] for item in speech_timestamps)
        if speech_timestamps and sum_talk_seconds > 1.0:
            chunks.append(chunk)
            seconds.append(start / sr)

    return np.stack(chunks), seconds

In [7]:
vad, utils = torch.hub.load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    force_reload=False,
    onnx=True
)

Using cache found in /home/demontego/.cache/torch/hub/snakers4_silero-vad_master


In [8]:
chunked_audio, seconds = chunk_audio_with_overlap(audio, vad, utils[0])

In [9]:
long_audio = np.vstack([chunked_audio] * 1)

In [10]:
long_audio.shape

(17, 160000)

In [11]:
proccessed_audio = processor.feature_extractor(long_audio, sampling_rate=16000, return_tensors="pt").input_features.to(torch.float16).cuda()

In [12]:
proccessed_audio.shape

torch.Size([17, 128, 3000])

In [14]:
results = model.generate(
    input_features=proccessed_audio,
    forced_decoder_ids=[[50258, 50360]],
    length_penalty=1.0,
    max_new_tokens=100,
    num_beams=3,
    condition_on_prev_tokens=True,
    # compression_ratio_threshold=1.35,
    temperature=0.1,
    return_dict_in_generate=True,
    return_timestamps=True)

In [40]:
def parse_whisper_output_proper(tokenizer_output, timestamps: list[float]) -> list[dict[str, float | str]]:
    """
    Правильный способ обработки через tokenizer Whisper
    """
    segments = []
    for tokens, timestamp in zip(tokenizer_output.numpy(), timestamps):
        # Используем встроенный метод декодирования
        # Этот метод уже есть в вашем процессоре
        decoded_segments, chunks = processor.tokenizer._decode_asr(
            [{"tokens": [tokens]}], 
            return_language=False,
            return_timestamps=True, 
            time_precision=0.02
        )
        for chunk in chunks['chunks']:
            segments.append({
                'start_time': chunk['timestamp'][0] + timestamp,
                'end_time': min(chunk['timestamp'][1], 10.0) + timestamp,
                'text': chunk['text'].strip()
            })
    
    return segments

In [41]:
parse_whisper_output_proper(results, seconds)

[{'start_time': 19.0,
  'end_time': 29.0,
  'text': 'Я читаю про дипсик и просто как индексфейли до этого не додумались просто'},
 {'start_time': 28.5,
  'end_time': 38.5,
  'text': 'просто нам прикольная прям архитектура что типа на поверхности что-то лежало там способ обучения типа он'},
 {'start_time': 38.0,
  'end_time': 48.0,
  'text': 'он много раз обсуждался, но почему-то я редко видел его в применении.'},
 {'start_time': 47.5, 'end_time': 48.42, 'text': 'в применении.'},
 {'start_time': 52.86,
  'end_time': 54.88,
  'text': 'А я знаю, почему они пошли'},
 {'start_time': 54.88, 'end_time': 57.5, 'text': 'по этому пути. Они начали'},
 {'start_time': 57.0,
  'end_time': 64.26,
  'text': 'начали обучать давно очень модель кодинга deepsick кодер не была лучшая модель вот и'},
 {'start_time': 64.26,
  'end_time': 67.0,
  'text': 'всех даже лучше чем сейчас gpt'},
 {'start_time': 66.5,
  'end_time': 75.14,
  'text': 'И они такие, надо еще лучше, и расширили пределы кодинга свою модель

In [45]:
results = processor.batch_decode(
    results,
    decode_with_timestamps=True,
    basic_normalize=True,
    skip_special_tokens=True,
    time_precision=0.02
)

In [44]:
import re

def parse_whisper_timestamps_simple(text: str) -> list[tuple[float, float, str]]:
    """
    Парсит простые случаи формата '<|start|> text <|end|>'
    """
    pattern = r'<\|(\d+\.?\d*)\|>\s*(.*?)\s*<\|(\d+\.?\d*)\|>'
    matches = re.findall(pattern, text)
    
    results = []
    for match in matches:
        start_time = float(match[0])
        end_time = float(match[2])
        content = match[1].strip()
        results.append((start_time, end_time, content))
    
    return results

In [ ]:
res

In [34]:
decoded, chunks = processor.tokenizer._decode_asr(
        [{"tokens": [results.flatten().numpy()]}], 
        return_timestamps=True, 
        return_language=False, 
        time_precision=0.02
    )
chunks

{'chunks': [{'timestamp': (0.0, 2.06), 'text': ' you'},
  {'timestamp': (0.0, 29.98), 'text': ' Продолжение следует...'},
  {'timestamp': (0.0, 10.0),
   'text': ' Я читаю про дипсик и просто как индексфейли до этого не додумались просто'},
  {'timestamp': (0.0, 10.0),
   'text': ' просто нам прикольная прям архитектура что типа на поверхности что-то лежало там способ обучения типа он'},
  {'timestamp': (0.0, 10.0),
   'text': ' он много раз обсуждался, но почему-то я редко видел его в применении.'},
  {'timestamp': (0.0, 0.92), 'text': ' в применении.'},
  {'timestamp': (5.34, 7.38), 'text': ' А я знаю, почему они пошли'},
  {'timestamp': (7.38, 10.0), 'text': ' по этому пути. Они начали'},
  {'timestamp': (0.0, 7.26),
   'text': ' начали обучать давно очень модель кодинга deepsick кодер не была лучшая модель вот и'},
  {'timestamp': (7.26, 10.0), 'text': ' всех даже лучше чем сейчас гтт'},
  {'timestamp': (0.0, 10.0),
   'text': ' и таки надо еще лучше и расширили пределы кодинга сво

In [50]:
chunks

{'chunks': [{'timestamp': (0.0, 1.0), 'text': ' пришел'},
  {'timestamp': (1.0, 6.06), 'text': ' я читаю'},
  {'timestamp': (6.06, 7.82), 'text': ' про дипсик и просто'},
  {'timestamp': (7.82, 10.0), 'text': ' как интересно'}]}

In [ ]:
result = pipe(audio, generate_kwargs=generate_kwargs, return_timestamps=True)
pprint(result)

In [1]:
import librosa

from src.core.pipe import Pipe
from src.core.config import ASRConfig, AudioProcessorConfig

config_asr = ASRConfig(model_id='openai/whisper-large-v3-turbo')
config_processor = AudioProcessorConfig()
pipe = Pipe(config_asr, config_processor)

pipe.warmup()

/home/demontego/whisper-on-speed/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /home/demontego/.cache/torch/hub/snakers4_silero-vad_master
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
audio, _ = librosa.load("/home/demontego/whisper/tests/test_data/output.wav", sr=16000)

In [6]:
result = pipe([audio])[0]

In [7]:
result

[Replica(text='Продолжение следует...', start_time=9.7, end_time=19.7),
 Replica(text='я читаю про дипсик и просто как индекс файлы до этого не додумались просто', start_time=19.4, end_time=29.4),
 Replica(text='Там прикольная прям архитектура, что на поверхности что-то лежало.', start_time=29.1, end_time=35.18),
 Replica(text='Там способ обучения, типа он...', start_time=35.980000000000004, end_time=39.1),
 Replica(text='много раз обсуждался', start_time=38.8, end_time=43.44),
 Replica(text='но почему-то', start_time=43.44, end_time=46.22),
 Replica(text='я редко видел его в применении', start_time=46.22, end_time=48.699999999999996),
 Replica(text='Я знаю, почему они пошли', start_time=48.5, end_time=54.88),
 Replica(text='по этому пути. Они', start_time=54.88, end_time=56.8),
 Replica(text='начали обучать давно', start_time=56.8, end_time=58.5),
 Replica(text='очень модель кодинга deepsick кодер не была лучшая модель вот и всех даже лучше чем сейчас гтт и надо', start_time=58.2, end